# Init

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.io import fits
from astropy.visualization import simple_norm
import photutils
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.stats import SigmaClip
from photutils.background import Background2D, MedianBackground
from astropy.stats import sigma_clipped_stats, SigmaClip
from photutils.segmentation import detect_threshold, detect_sources
from photutils.utils import circular_footprint
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.segmentation import detect_sources
from scipy import stats
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.wcs import WCS
from photutils.detection import DAOStarFinder
from astroquery.gaia import Gaia
import pandas as pd
import subprocess
import shutil
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from pathlib import Path
from scipy.stats import siegelslopes

# Load files

In [3]:

observations_folder = r"/mnt/c/Users/catal/Downloads/unistellar_observations"

# List of all folders observation
observation_dirs = [d for d in os.listdir(observations_folder) 
                    if os.path.isdir(os.path.join(observations_folder, d))]
observation_dirs.sort()

print(f"Encontradas {len(observation_dirs)} observaciones para procesar\n")
print("=" * 60)


total_processed = 0
total_errors = 0

for obs_idx, obs_dir in enumerate(observation_dirs, 1):
    base_folder = os.path.join(observations_folder, obs_dir)
    dark_subtracted_folder = os.path.join(base_folder, "dark_subtracted")
    green_band_folder = os.path.join(base_folder, "green_band")
    
    # Verification if the folder exists
    if not os.path.exists(dark_subtracted_folder):
        print(f"[{obs_idx}/{len(observation_dirs)}] {obs_dir}")
        print(f"  ⚠ No tiene carpeta 'dark_subtracted', saltando...\n")
        continue
    
    print(f"[{obs_idx}/{len(observation_dirs)}] {obs_dir}")
    
    # Creation of new folder
    os.makedirs(green_band_folder, exist_ok=True)
    
    # Search for all .fits images
    fits_files = [f for f in os.listdir(dark_subtracted_folder) if f.endswith('.fits')]
    print(f"  Encontradas {len(fits_files)} imágenes")
    
    # Image processing
    for i, fits_file in enumerate(fits_files, 1):
        file_path = os.path.join(dark_subtracted_folder, fits_file)
        print(f"    [{i}/{len(fits_files)}] {fits_file}...", end=" ")
        
        try:
            # Opensfits file
            with fits.open(file_path) as hdul:
                header = hdul[0].header
                data = hdul[0].data.astype(float)
            
            wcs = WCS(header)
            
            # --------------------------------------------------
            # Green pattern (Bayern pattern RGGB)
            # Pixels in G band: (even row, odd col) y (odd row, even col)
            # --------------------------------------------------
            G = (data[0::2, 1::2] + data[1::2, 0::2]) / 2.0
            
            # --------------------------------------------------
            # FIX WCS (image size reduced by factor 2)
            # --------------------------------------------------
            wcs_green = wcs.deepcopy()
            
            if wcs_green.wcs.crpix is not None:
                wcs_green.wcs.crpix /= 2.0
            
            if wcs_green.wcs.cdelt is not None:
                wcs_green.wcs.cdelt *= 2.0
            
            # Update WCS keywords in the original header
            header.update(wcs_green.to_header())
            
            # Save image with green filter
            base_name = os.path.splitext(fits_file)[0]
            new_filename = os.path.join(green_band_folder, base_name + "_g_band.fits")
            
            hdu = fits.PrimaryHDU(G, header=header)
            hdu.writeto(new_filename, overwrite=True)
            
            print("✓")
            total_processed += 1
            
        except Exception as e:
            print(f"✗ Error: {str(e)}")
            total_errors += 1
    
    print()

print("=" * 60)
print(f"Procesamiento completado")
print(f"Total procesadas: {total_processed}")
print(f"Total errores: {total_errors}")


Encontradas 11 observaciones para procesar

[1/11] 20260202T180641_707
  Encontradas 23 imágenes
    [1/23] 20260202T180649_887_StackInput_DarkSubtracted.fits... ✓
    [2/23] 20260202T180653_859_StackInput_DarkSubtracted.fits... ✓
    [3/23] 20260202T180653_859_StackInput_DarkSubtracted_g_band.fits... ✓
    [4/23] 20260202T180657_831_StackInput_DarkSubtracted.fits... ✓
    [5/23] 20260202T180701_803_StackInput_DarkSubtracted.fits... ✓
    [6/23] 20260202T180705_775_StackInput_DarkSubtracted.fits... ✓
    [7/23] 20260202T180709_746_StackInput_DarkSubtracted.fits... ✓
    [8/23] 20260202T180713_718_StackInput_DarkSubtracted.fits... ✓
    [9/23] 20260202T180717_690_StackInput_DarkSubtracted.fits... ✓
    [10/23] 20260202T180721_662_StackInput_DarkSubtracted.fits... ✓
    [11/23] 20260202T180725_634_StackInput_DarkSubtracted.fits... ✓
    [12/23] 20260202T180729_605_StackInput_DarkSubtracted.fits... ✓
    [13/23] 20260202T180733_577_StackInput_DarkSubtracted.fits... ✓
    [14/23] 20260202T

## Red Band Extraction (RGGB Bayer Pattern)

In [ ]:
observations_folder = r"/mnt/c/Users/catal/Downloads/unistellar_observations"

# List of all folders observation
observation_dirs = [d for d in os.listdir(observations_folder) 
                    if os.path.isdir(os.path.join(observations_folder, d))]
observation_dirs.sort()

print(f"Encontradas {len(observation_dirs)} observaciones para procesar\n")
print("=" * 60)


total_processed = 0
total_errors = 0

for obs_idx, obs_dir in enumerate(observation_dirs, 1):
    base_folder = os.path.join(observations_folder, obs_dir)
    dark_subtracted_folder = os.path.join(base_folder, "dark_subtracted")
    red_band_folder = os.path.join(base_folder, "red_band")
    
    # Verification if the folder exists
    if not os.path.exists(dark_subtracted_folder):
        print(f"[{obs_idx}/{len(observation_dirs)}] {obs_dir}")
        print(f"  ⚠ No tiene carpeta 'dark_subtracted', saltando...\n")
        continue
    
    print(f"[{obs_idx}/{len(observation_dirs)}] {obs_dir}")
    
    # Creation of new folder
    os.makedirs(red_band_folder, exist_ok=True)
    
    # Search for all .fits images
    fits_files = [f for f in os.listdir(dark_subtracted_folder) if f.endswith('.fits')]
    print(f"  Encontradas {len(fits_files)} imágenes")
    
    # Image processing
    for i, fits_file in enumerate(fits_files, 1):
        file_path = os.path.join(dark_subtracted_folder, fits_file)
        print(f"    [{i}/{len(fits_files)}] {fits_file}...", end=" ")
        
        try:
            # Open fits file
            with fits.open(file_path) as hdul:
                header = hdul[0].header
                data = hdul[0].data.astype(float)
            
            wcs = WCS(header)
            
            # --------------------------------------------------
            # Red pattern (Bayern pattern RGGB)
            # Red pixels: (even row, even col)
            # --------------------------------------------------
            R = data[0::2, 0::2]
            
            # --------------------------------------------------
            # FIX WCS (image size reduced by factor 2)
            # --------------------------------------------------
            wcs_red = wcs.deepcopy()
            
            if wcs_red.wcs.crpix is not None:
                wcs_red.wcs.crpix /= 2.0
            
            if wcs_red.wcs.cdelt is not None:
                wcs_red.wcs.cdelt *= 2.0
            
            # Update WCS keywords in the original header
            header.update(wcs_red.to_header())
            
            # Save image with red filter
            base_name = os.path.splitext(fits_file)[0]
            new_filename = os.path.join(red_band_folder, base_name + "_r_band.fits")
            
            hdu = fits.PrimaryHDU(R, header=header)
            hdu.writeto(new_filename, overwrite=True)
            
            print("✓")
            total_processed += 1
            
        except Exception as e:
            print(f"✗ Error: {str(e)}")
            total_errors += 1
    
    print()

print("=" * 60)
print(f"Procesamiento completado")
print(f"Total procesadas: {total_processed}")
print(f"Total errores: {total_errors}")

## Blue Band Extraction (RGGB Bayer Pattern)

In [ ]:
observations_folder = r"/mnt/c/Users/catal/Downloads/unistellar_observations"

# List of all folders observation
observation_dirs = [d for d in os.listdir(observations_folder) 
                    if os.path.isdir(os.path.join(observations_folder, d))]
observation_dirs.sort()

print(f"Encontradas {len(observation_dirs)} observaciones para procesar\n")
print("=" * 60)


total_processed = 0
total_errors = 0

for obs_idx, obs_dir in enumerate(observation_dirs, 1):
    base_folder = os.path.join(observations_folder, obs_dir)
    dark_subtracted_folder = os.path.join(base_folder, "dark_subtracted")
    blue_band_folder = os.path.join(base_folder, "blue_band")
    
    # Verification if the folder exists
    if not os.path.exists(dark_subtracted_folder):
        print(f"[{obs_idx}/{len(observation_dirs)}] {obs_dir}")
        print(f"  ⚠ No tiene carpeta 'dark_subtracted', saltando...\n")
        continue
    
    print(f"[{obs_idx}/{len(observation_dirs)}] {obs_dir}")
    
    # Creation of new folder
    os.makedirs(blue_band_folder, exist_ok=True)
    
    # Search for all .fits images
    fits_files = [f for f in os.listdir(dark_subtracted_folder) if f.endswith('.fits')]
    print(f"  Encontradas {len(fits_files)} imágenes")
    
    # Image processing
    for i, fits_file in enumerate(fits_files, 1):
        file_path = os.path.join(dark_subtracted_folder, fits_file)
        print(f"    [{i}/{len(fits_files)}] {fits_file}...", end=" ")
        
        try:
            # Open fits file
            with fits.open(file_path) as hdul:
                header = hdul[0].header
                data = hdul[0].data.astype(float)
            
            wcs = WCS(header)
            
            # --------------------------------------------------
            # Blue pattern (Bayern pattern RGGB)
            # Blue pixels: (odd row, odd col)
            # --------------------------------------------------
            B = data[1::2, 1::2]
            
            # --------------------------------------------------
            # FIX WCS (image size reduced by factor 2)
            # --------------------------------------------------
            wcs_blue = wcs.deepcopy()
            
            if wcs_blue.wcs.crpix is not None:
                wcs_blue.wcs.crpix /= 2.0
            
            if wcs_blue.wcs.cdelt is not None:
                wcs_blue.wcs.cdelt *= 2.0
            
            # Update WCS keywords in the original header
            header.update(wcs_blue.to_header())
            
            # Save image with blue filter
            base_name = os.path.splitext(fits_file)[0]
            new_filename = os.path.join(blue_band_folder, base_name + "_b_band.fits")
            
            hdu = fits.PrimaryHDU(B, header=header)
            hdu.writeto(new_filename, overwrite=True)
            
            print("✓")
            total_processed += 1
            
        except Exception as e:
            print(f"✗ Error: {str(e)}")
            total_errors += 1
    
    print()

print("=" * 60)
print(f"Procesamiento completado")
print(f"Total procesadas: {total_processed}")
print(f"Total errores: {total_errors}")